In [42]:
import pandas as pd
import numpy as np
import pymysql
import psycopg2
from bs4 import BeautifulSoup
import requests
import yaml

In [43]:
import configparser
config = configparser.ConfigParser()
config.read('config.ini')
config.sections()

['SOS_DB', 'docdx_db_production']

In [44]:
sos_db_section = config['SOS_DB']
host = sos_db_section['host']
user = sos_db_section['user']
password = sos_db_section['password']
db=sos_db_section['db']
port = int(sos_db_section.get('port', 3306))
db = pymysql.connect(host=host, user=user, password=password, db=db, port=port)

import_db_section = config['docdx_db_production']
host = import_db_section['host']
user = import_db_section['user']
password = import_db_section['password']
db_dx=import_db_section['db']
port = int(sos_db_section.get('port',5432))

conn_str = "host={} dbname={} user={} password={}".format(host, db_dx, user, password)
db_docdx = psycopg2.connect(conn_str)

In [15]:
#get all users who ever generate magic_link 
query = """SELECT distinct m.user_id, up.npi FROM private.magic_link m
JOIN public."user_profile" up
ON m.user_id = up.user_id
where npi is not null and m.used_at is not null """
all_ml_npi = pd.read_sql(query, db_docdx)

In [16]:
all_ml_npi

,user_id,npi
0,472046,1629011846
1,472064,1659374940
2,472080,1437162716
3,472090,1649238841
4,472110,1437131109
5,472137,1124096136
6,472148,1124031075
7,472152,1659310704
8,472154,1184611634
9,472158,1952403446


In [ ]:
##web crawler

In [ ]:
#import urllib.request

#user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

In [31]:
loc = dict()
for idx, row in all_ml_npi.iterrows():
    try:
        npi = int(row['npi'])
        url = 'https://npiregistry.cms.hhs.gov/api/?number=' + str(npi)
        response = requests.get(url)
        page = BeautifulSoup(response.content,"html.parser")
        s = page.text
        #dic = yaml.load(s)
        #print(page)
        city = yaml.load(s)['results'][0]['addresses'][0]['city']
        state = yaml.load(s)['results'][0]['addresses'][0]['state']
        post_code = yaml.load(s)['results'][0]['addresses'][0]['postal_code']
        loc[npi] = [city, state, post_code]
    except Exception:
        pass

In [32]:
loc

{1629011846: ['SAINT LOUIS', 'MO', '631366132'],
 1659374940: ['RENO', 'NV', '895094857'],
 1437162716: ['TYLER', 'TX', '75701'],
 1649238841: ['REISTERSTOWN', 'MD', '211362515'],
 1437131109: ['OAK RIDGE', 'TN', '378306471'],
 1124096136: ['KNOXVILLE', 'TN', '379195706'],
 1124031075: ['BRISTOL', 'CT', '060104068'],
 1659310704: ['MEDFORD', 'OR', '97504'],
 1184611634: ['SOMERSET', 'PA', '155012104'],
 1952403446: ['CHARLESTON', 'SC', '294077174'],
 1255340675: ['YAKIMA', 'WA', '989081661'],
 1942525027: ['NORTH VERNON', 'IN', '472651030'],
 1184650848: ['GLASSBORO', 'NJ', '080281403'],
 1174517916: ['CHESTER', 'NJ', '079302918'],
 1629033378: ['YUMA', 'AZ', '853646912'],
 1356671515: ['HOUSTON', 'TX', '770742073'],
 1285770123: ['YARDLEY', 'PA', '190677704'],
 1285714600: ['HOUSTON', 'TX', '770302316'],
 1790947232: ['BRONX', 'NY', '104612720'],
 1093911588: ['SAXONBURG', 'PA', '160562254'],
 1356314751: ['TOMS RIVER', 'NJ', '087532975'],
 1235172768: ['FORT MYERS', 'FL', '339074539'

In [33]:
df_loc = pd.DataFrame.from_dict(loc, orient = 'index')
df_loc.columns = ['City','State','Zip']

In [34]:
df_loc = df_loc.reset_index()
df_loc.columns = ['npi','City','State','Zip']

In [36]:
df_loc.to_excel('user_loc.xlsx', index = False)

In [51]:
user_loc_dict = dict(zip(df_loc.npi, df_loc.State))

In [41]:
df_loc.groupby('State').count()[['npi']].sort_values('npi', ascending = False)#.to_clipboard()

In [49]:
#whoever answered a question 
query_comment = """select distinct c.created_by, spe.name spe, sub.name subspe, up.npi
	from comment c
		JOIN public."user_profile" up
		ON c.created_by = up.user_id
		JOIN specialty spe
		ON spe.id = up.specialty_id
		JOIN subspecialty sub
		on sub.id = up.subspecialty_id
        where npi is not null"""
comment = pd.read_sql(query_comment, db_docdx)

In [59]:
comment['state'] = comment.npi.apply(lambda x: user_loc_dict[x] if x in user_loc_dict.keys() else None)

In [60]:
comment

,created_by,spe,subspe,npi,state
0,472080,Gastroenterology,Gastroenterology,1437162716,TX
1,472110,Family Practice,Family Medicine,1437131109,TN
2,472152,Internal Medicine,Internal Medicine,1659310704,OR
3,472154,Internal Medicine,Internal Medicine,1184611634,PA
4,472158,Dermatology,Dermatology,1952403446,SC
5,472177,Family Practice,Family Medicine,1255340675,WA
6,472199,Family Practice,Family Medicine,1184650848,NJ
7,472353,Cardiology,Clinical Cardiac Electrophysiology,1235172768,FL
8,472493,Family Practice,Family Medicine,1316258080,PA
9,472502,Pulmonology,Pulmonary Critical Care Medicine,1235386517,NY


In [63]:
comment.groupby('state').count()[['npi']].sort_values('npi', ascending = False).to_clipboard()

In [68]:
comment.groupby(['state','spe']).count()[['npi']].reset_index()#.to_clipboard()#.sort_values('npi', ascending = False)

In [69]:
query_mg_spe_npi = """SELECT distinct m.user_id, spe.name spe, sub.name subspe, up.npi FROM private.magic_link m
JOIN public."user_profile" up
ON m.user_id = up.user_id
		JOIN specialty spe
		ON spe.id = up.specialty_id
		JOIN subspecialty sub
		on sub.id = up.subspecialty_id
where npi is not null and m.used_at is not null"""
ml_npi_spe = pd.read_sql(query_mg_spe_npi, db_docdx)

In [71]:
ml_npi_spe['state'] = ml_npi_spe.npi.apply(lambda x: user_loc_dict[x] if x in user_loc_dict.keys() else None)
#comment['state'] = comment.npi.apply(lambda x: user_loc_dict[x] if x in user_loc_dict.keys() else None)

In [74]:
ml_npi_spe.groupby(['state','spe']).count()[['npi']].reset_index().to_clipboard()